<a href="https://colab.research.google.com/github/Bhuvanesh-Singla/software-defect-detection/blob/main/1dCNN_version1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, chi2

In [2]:
!wget https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/jm1.arff
!wget https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/cm1.arff

--2023-10-29 11:06:30--  https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/jm1.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 861589 (841K) [text/plain]
Saving to: ‘jm1.arff’

jm1.arff            100%[===================>] 841.40K  --.-KB/s    in 0.007s  

2023-10-29 11:06:30 (112 MB/s) - ‘jm1.arff’ saved [861589/861589]

--2023-10-29 11:06:30--  https://raw.githubusercontent.com/Bhuvanesh-Singla/software-defect-detection/main/cm1.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56758 (55K

In [3]:
from scipy.io import arff
data = arff.loadarff('jm1.arff')
df= pd.DataFrame(data[0])
df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,b'false'
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,b'true'
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51.0,10.0,8.0,1.0,17.0,36.0,112.0,86.0,13.0,b'true'
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129.0,29.0,28.0,2.0,17.0,135.0,329.0,271.0,5.0,b'true'
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28.0,1.0,6.0,0.0,11.0,16.0,76.0,50.0,7.0,b'true'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,1.0,4.0,52.0,241.48,0.14,7.33,32.93,1770.86,...,13.0,0.0,2.0,0.0,10.0,15.0,30.0,22.0,7.0,b'false'
10881,9.0,2.0,1.0,2.0,30.0,129.66,0.12,8.25,15.72,1069.68,...,5.0,0.0,2.0,0.0,12.0,8.0,19.0,11.0,3.0,b'false'
10882,42.0,4.0,1.0,2.0,103.0,519.57,0.04,26.40,19.68,13716.72,...,29.0,1.0,10.0,0.0,18.0,15.0,59.0,44.0,7.0,b'false'
10883,10.0,1.0,1.0,1.0,36.0,147.15,0.12,8.44,17.44,1241.57,...,6.0,0.0,2.0,0.0,9.0,8.0,21.0,15.0,1.0,b'false'


In [4]:
df.defects.replace({b'false': 0 ,b'true': 1}, inplace = True)

In [5]:
df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,0
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51.0,10.0,8.0,1.0,17.0,36.0,112.0,86.0,13.0,1
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129.0,29.0,28.0,2.0,17.0,135.0,329.0,271.0,5.0,1
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28.0,1.0,6.0,0.0,11.0,16.0,76.0,50.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,1.0,4.0,52.0,241.48,0.14,7.33,32.93,1770.86,...,13.0,0.0,2.0,0.0,10.0,15.0,30.0,22.0,7.0,0
10881,9.0,2.0,1.0,2.0,30.0,129.66,0.12,8.25,15.72,1069.68,...,5.0,0.0,2.0,0.0,12.0,8.0,19.0,11.0,3.0,0
10882,42.0,4.0,1.0,2.0,103.0,519.57,0.04,26.40,19.68,13716.72,...,29.0,1.0,10.0,0.0,18.0,15.0,59.0,44.0,7.0,0
10883,10.0,1.0,1.0,1.0,36.0,147.15,0.12,8.44,17.44,1241.57,...,6.0,0.0,2.0,0.0,9.0,8.0,21.0,15.0,1.0,0


In [6]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy = 'mean').set_output(transform='pandas')
si.fit(df)
df = si.transform(df)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
selector = SelectPercentile(chi2, percentile=80)
selector.fit(df, df['defects'])
mask = selector.get_support()
features = df.iloc[:,mask]

In [10]:
features

,loc,v(g),iv(g),n,v,d,i,e,t,lOCode,lOComment,lOBlank,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,2.0,2.0,2.0,1.2,1.2,1.2,1.4,0.0
1,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,72.0,7.0,6.0,198.0,1134.13,20.31,55.85,23029.10,1279.39,51.0,10.0,8.0,36.0,112.0,86.0,13.0,1.0
3,190.0,3.0,3.0,600.0,4348.76,17.06,254.87,74202.67,4122.37,129.0,29.0,28.0,135.0,329.0,271.0,5.0,1.0
4,37.0,4.0,4.0,126.0,599.12,17.19,34.86,10297.30,572.07,28.0,1.0,6.0,16.0,76.0,50.0,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,4.0,52.0,241.48,7.33,32.93,1770.86,98.38,13.0,0.0,2.0,15.0,30.0,22.0,7.0,0.0
10881,9.0,2.0,2.0,30.0,129.66,8.25,15.72,1069.68,59.43,5.0,0.0,2.0,8.0,19.0,11.0,3.0,0.0
10882,42.0,4.0,2.0,103.0,519.57,26.40,19.68,13716.72,762.04,29.0,1.0,10.0,15.0,59.0,44.0,7.0,0.0
10883,10.0,1.0,1.0,36.0,147.15,8.44,17.44,1241.57,68.98,6.0,0.0,2.0,8.0,21.0,15.0,1.0,0.0


In [11]:
scaler = StandardScaler()

In [12]:
X = features.drop('defects', axis = 1)
y = features.defects

In [13]:
X = scaler.fit_transform(X)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [15]:
X_train.shape

(8708, 16)

In [16]:
#The first CNN layer consists of 32 filters with a kernel size of 2 along with ReLU as an activation function
#It is then followed by another convolution layer consisting of 16 filters with a kernel size of 2.
#dropout with a 0.5
#max-pooling having a pool size of 2.
#flatten -> 100, 75, 50, 25,2

In [17]:
model = Sequential()
model.add(Conv1D(32, 2, activation = 'relu', input_shape = (16,1)))
model.add(Conv1D(16,2, activation = 'relu'))
model.add(Conv1D(16,2, activation = 'relu'))
model.add(Conv1D(16,2, activation = 'relu'))
model.add(Conv1D(16,2, activation = 'relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(2))

model.add(Flatten())

model.add(Dense(100, activation = 'relu'))
model.add(Dense(75, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))


In [18]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )

In [19]:
history = model.fit(X_train, y_train, epochs = 100, validation_data= (X_val, y_val))

Epoch 1/100
273/273 [==============================] - 9s 11ms/step - loss: 0.4785 - accuracy: 0.8055 - val_loss: 0.4566 - val_accuracy: 0.8130
Epoch 2/100
273/273 [==============================] - 3s 9ms/step - loss: 0.4581 - accuracy: 0.8074 - val_loss: 0.4529 - val_accuracy: 0.8126
Epoch 3/100
273/273 [==============================] - 4s 14ms/step - loss: 0.4531 - accuracy: 0.8076 - val_loss: 0.4493 - val_accuracy: 0.8103
Epoch 4/100
273/273 [==============================] - 3s 10ms/step - loss: 0.4515 - accuracy: 0.8075 - val_loss: 0.4470 - val_accuracy: 0.8107
Epoch 5/100
273/273 [==============================] - 3s 9ms/step - loss: 0.4515 - accuracy: 0.8079 - val_loss: 0.4446 - val_accuracy: 0.8126
Epoch 6/100
273/273 [==============================] - 2s 7ms/step - loss: 0.4502 - accuracy: 0.8103 - val_loss: 0.4409 - val_accuracy: 0.8149
Epoch 7/100
273/273 [==============================] - 1s 4ms/step - loss: 0.4474 - accuracy: 0.8087 - val_loss: 0.4408 - val_accuracy: 0.8

In [20]:
test_data = arff.loadarff('cm1.arff')
test_df= pd.DataFrame(test_data[0])
test_df

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,b'false'
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,b'true'
2,24.0,5.0,1.0,3.0,63.0,309.13,0.11,9.50,32.54,2936.77,...,1.0,0.0,6.0,0.0,15.0,15.0,44.0,19.0,9.0,b'false'
3,20.0,4.0,4.0,2.0,47.0,215.49,0.06,16.00,13.47,3447.89,...,0.0,0.0,3.0,0.0,16.0,8.0,31.0,16.0,7.0,b'false'
4,24.0,6.0,6.0,2.0,72.0,346.13,0.06,17.33,19.97,5999.58,...,0.0,0.0,3.0,0.0,16.0,12.0,46.0,26.0,11.0,b'false'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,47.0,3.0,1.0,3.0,256.0,1563.78,0.04,28.00,55.85,43785.90,...,2.0,13.0,2.0,0.0,23.0,46.0,144.0,112.0,5.0,b'true'
494,24.0,4.0,3.0,3.0,107.0,587.63,0.05,19.13,30.72,11241.58,...,1.0,7.0,4.0,0.0,22.0,23.0,67.0,40.0,7.0,b'true'
495,82.0,11.0,3.0,10.0,475.0,3155.83,0.02,44.71,70.59,141084.24,...,9.0,59.0,35.0,0.0,32.0,68.0,285.0,190.0,21.0,b'true'
496,10.0,2.0,1.0,1.0,32.0,150.41,0.15,6.50,23.14,977.69,...,1.0,12.0,4.0,0.0,13.0,13.0,19.0,13.0,3.0,b'true'


In [21]:
test_df.defects.replace({b'false': 0 ,b'true': 1}, inplace = True)

In [22]:
selector = SelectPercentile(chi2, percentile=80)
selector.fit(test_df, test_df['defects'])
mask = selector.get_support()
test_features = test_df.iloc[:,mask]

In [23]:
test_features

,loc,v(g),iv(g),n,v,d,i,e,t,lOComment,lOBlank,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,2.0,2.0,1.2,1.2,1.2,1.2,1.4,0
1,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,24.0,5.0,3.0,63.0,309.13,9.50,32.54,2936.77,163.15,0.0,6.0,15.0,15.0,44.0,19.0,9.0,0
3,20.0,4.0,2.0,47.0,215.49,16.00,13.47,3447.89,191.55,0.0,3.0,16.0,8.0,31.0,16.0,7.0,0
4,24.0,6.0,2.0,72.0,346.13,17.33,19.97,5999.58,333.31,0.0,3.0,16.0,12.0,46.0,26.0,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,47.0,3.0,3.0,256.0,1563.78,28.00,55.85,43785.90,2432.55,13.0,2.0,23.0,46.0,144.0,112.0,5.0,1
494,24.0,4.0,3.0,107.0,587.63,19.13,30.72,11241.58,624.53,7.0,4.0,22.0,23.0,67.0,40.0,7.0,1
495,82.0,11.0,10.0,475.0,3155.83,44.71,70.59,141084.24,7838.01,59.0,35.0,32.0,68.0,285.0,190.0,21.0,1
496,10.0,2.0,1.0,32.0,150.41,6.50,23.14,977.69,54.32,12.0,4.0,13.0,13.0,19.0,13.0,3.0,1


In [24]:
X_test = test_features.drop('defects', axis = 1)
y_test = test_features.defects

In [25]:
X_test = scaler.fit_transform(X_test)

In [28]:
model.evaluate(X_test, y_test)

16/16 [==============================] - 0s 5ms/step - loss: 0.3611 - accuracy: 0.8735


[0.36113035678863525, 0.8734939694404602]